❗️ **If there is a problem rendering this article (garbled or incorrect format), please feel free to visit my Github repository for this course:**

https://github.com/jimcui3/Introduction-to-Machine-Learning

# *Introduction to Machine Learning* Algorithms and Realizations 4.1

## By Jiaheng Cui
>In chapter 4, we'll talk about Decision Trees, a widely-used model in supervised learning. We'll do a basic introduction in this passage, and introduce an early algorithm to generate a decision tree: ID3. In the next passage, we'll introduce C4.5 and CART algorithms, and prunning techniques.

## 0.Tips:
### (1) Prerequisites:
For those who hasn't learned **graph theory** or **data stuctures**, please have a brief understanding of graphs and trees by taking a look at these two pages:

  1.Graph: https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)

  2.Tree: https://en.wikipedia.org/wiki/Tree_(graph_theory)

### (2) Decision tree from sklearn:
If you want to use decision tree algorithms from **sklearn**, please note that they only accept numerical values, but not categorical values. So you need to first transform the categorical values into numerical values using `preprocessing.LabelEncoder()`.

References: 
1. Decision tree from sklearn: https://scikit-learn.org/stable/modules/tree.html

2. LabelEncoder from sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

3. Using LabelEncoder to transform categorical variables: https://stackoverflow.com/questions/38108832/passing-categorical-data-to-sklearn-decision-tree

## 1.Decision Trees
### (1) Introduction
Decision tree is a tree-like model that can both perform classification and regression tasks. It could be regarded as a set of if-then statements. Every decision tree is made up of nodes and **directed** edges, where the internal nodes represent attributes and the leaf nodes represent labels.

Every path from the root node to a leaf node represent a classification for a certain sample. Starting from the root node, the model examines whether the sample follows which if-then statement, then pass it along that edge, iterate this procedure and finally reach a leaf node, which represent its label.

### (2) Building a tree
The basic thought of building a decision tree is listed as follows:

(1) Pick a feature selection method for the tree.

(2) Use the feature selection method to examine the root node and create several if-then statements, each statement extends an edge linking to a child node.

(3) Divide the original dataset to several subsets according to the statements, and pass them to child nodes.

(4) If there exist a subset that is pure (i.e. has only one output result), then make it a leaf node; otherwise, continue using the feature selection method to examine the child nodes..

(5) The building process will end if all paths lead to a leaf node.

But how to decide which attribute is in each node, and what is the threshold for every if-then statement? We need to find out some feature selection methods. There are three well-known feature selection methods: information gain, information gain ratio and Gini index. These tree methods lead to three decision tree algorithms: ID3, C4.5 and CART.

Among the three, the simplest but also the roughest is the ID3 algorithm.

## 2.ID3 Algorithm
### (1) Entropy
Before introducing information gain, we need to know how to describe how much information that a random variable could convey. This leads to a important concept in Information Theory: Entropy.

Let $X$ be a random variable taking values in the alphabet $\mathcal{X} = \left \{ x_1, x_2, ..., x_n \right \}$, and follows the probability distribution $p(x) = \left \{ p_1, p_2, ..., p_n \right \}$. Then the entropy of $X$, denoted as $H(X)$, is:
$$H(X) = \sum_{x \in \mathcal{X}} p(x) log {\frac{1}{p(x)}} = - \sum_{i = 1}^{n} p(x_i) log {p(x_i)}$$

Here $log$ is the logarithm to base 2, i.e. $log_2$. The commonly used values include $log 2 = 1$, $log 3 = 1.585$, $log 5 = 2.322$, $log 7 = 2.807$.

We want to define the entropy of sets. Consider the dataset $D$. $D$ has $\mid D \mid$ samples, where $\mid \bullet \mid$ is the cardinal of a set.

The label of $D$ takes values in $\mathcal{D} = \left \{ d_1, d_2, ..., d_n \right \}$, the corresponding subsets are denoted as $C_1, C_2, ..., C_n$, i.e. $C_i = \left \{ x \in D: y = d_i \right \}$, $i = 1, 2, ..., n$.

* For example, $\mathcal{D}$ could be $\left \{ "yes", "no" \right \}$, then we can devide D into $C_1 = \left \{ x \in D: y = "yes" \right \}$ and $C_2 = \left \{ x \in D: y = "no" \right \}$.

Since $\sum_{i = 1}^{n} \mid C_i \mid = \mid D \mid$, and we can estimate the proportions of each label $p_i$: $\hat{p_i} = \frac{\mid C_i \mid}{\mid D \mid}$, $i = 1, 2, ..., n$.

Then we can define the entropy of $D$ as $H(D) = - \sum_{i = 1}^{n} \frac{\mid C_i \mid}{\mid D \mid} log \frac{\mid C_i \mid}{\mid D \mid}$.

### (2) Information gain
Suppose we have an attribute $A$ taking possible values in $\mathcal{A} = \left \{ a_1, a_2, ..., a_m \right \}$. Let $D_i = \left \{ x \in D: x.A = a_i \right \}$, $i = 1, 2, ..., m$, where $x.A$ is the value of $x$ in attribute $A$.

* For example, $A$ could represent the weather, and $\mathcal{A}$ may be $\left \{ "sunny", "rainy", "cloudy", "snowy" \right \}$, $D_1 = \left \{ x \in D: x.weather = "sunny" \right \}$.

Then the conditional entropy of $D$ conditioned on $A$ is $H(D \mid A) = \sum_{i = 1}^{m} \frac{\mid D_i \mid}{\mid D \mid} H(D_i)$.

Finally, the information gain of $D$ for attribute $A$ is $Gain(D, A) = H(D) - H(D \mid A)$.

### (3) ID3 Algorithm
❗️ There are recursions in the algorithm, i.e. the function may call itself during computing. It may be a little difficult to understand the recursion, if so, please review the following algorithm carefully and make sure you understand it clearly!

Input: training set $D = \left \{(X, y) \right \}$, where $X$ is the dataset of feature and $y$ is the label vector, the attribute set $\mathbb{A} = \left \{ A_1, A_2, ..., A_k\right \}$, where $A_i$ is an attribute, $i = 1, 2, ..., k$.

Output: the decision tree $T$.

Step1: load the **current** training set $D$ and **current** attribute set $\mathbb{A}$. 

Step2: if $D$ has only one type of output values, i.e. there exists a $C_i$ s.t. $D = C_i$, then $T$ has a single node, and it's a leaf node. The label of this node is $c_i$. Return $T$.

Step3: if $\mathbb{A}$ is empty, the building process is over, return $T$.

Step4: suppose $\mathbb{A} = \left \{ A_1, A_2, ..., A_k\right \}$ now. Compute the information gain of $D$ for all the attributes, i.e. $Gain(D, A_i)$, $i = 1, 2, ..., k$.

Step5: find the attribute which has the **largest** information gain, denote it as $A_g = \left \{ a_{g1}, a_{g2}, ..., a_{gm} \right \}$, let the attribute of the current node be $A_g$.

Step6: divide $D$ into $D_1, D_2, ..., D_m$, where $D_i = \left \{ x \in D: x.A_g = a_{gi}\right \}$. For each $D_i$, create a child node of the current node, and pass $D_i$ to it as the training set.

Step7: for the i-th child node, **recursively call Steps1-6** with training set $D_i$ and attribute set $\mathbb{A} - \left \{ A_g \right \}$, the result of the recursion is denoted as $T_i$, $T_i$ is the subtree of $T$ with this child node as the root node.


### (4) Code

In [1]:
import numpy as np
import pandas as pd
from numpy import log2 as log # "log" is the logarithm to base 2

In [2]:
class ID3_Classification_Tree():
    def __init__(self, df):
        self.df = df
        self.label = list(df)
     
    
    # Calculate the entropy of different attributes
    def find_entropy(self, df):
        entropy = 0
        Class = df.keys()[-1] # self.df.keys()[-1] is the last column, i.e. the label
        values = df[Class].unique()# We want to devide df by the label, so we find the distinct labels
        
        for value in values:
            fraction = df[Class].value_counts()[value]/len(df[Class])# fraction_i = |C_k|/|D|
            
            if(fraction != 0): # Let 0*log0 = 0, otherwise we cannot compute the entropy
                entropy += -fraction * log(fraction)# entropy_i = -|C_k|/|D|* log(|C_k|/|D|)
                
        return entropy

    
    # Calculate the entropy within an attribute (i.e. subsets)
    def find_entropy_attribute(self, df, attribute):
        Class = df.keys()[-1]
        target_variables = df[Class].unique()
        variables = df[attribute].unique() # Find the distinct features within the attribute
        conditional_entropy = 0
        
        for variable in variables:
            entropy = 0
            
            for target_variable in target_variables:
                num = len(df[attribute][df[attribute] == variable][df[Class] == target_variable]) # |D_ij|
                den = len(df[attribute][df[attribute] == variable]) # |D_i|
                fraction = num/den # |D_ij|/|D_i|
                
                if(fraction != 0):
                    entropy += -fraction * log(fraction) # entropy is the entropy of one of the values in the attribute
                    
            fraction2 = den/len(df) # |D_i|/|D|
            conditional_entropy += fraction2 * entropy # conditional_entropy is the conditional entropy under the attribute
            
        return conditional_entropy
    
    
    # Find the attribute that maximizes the information gain
    def find_winner(self, df): 
        competitors = [] # competitors is all possible moves of an node
        
        for key in df.keys()[:-1]: # df.keys()[:-1] is all the attributes of df except for the label
            competitors.append(self.find_entropy(df) - self.find_entropy_attribute(df, key)) # Compute the information gain of key
            
        return df.keys()[:-1][np.argmax(competitors)] 

    
    # Get the subset of a node by its value
    def get_subtable(self, df, node, value):
        return df[df[node] == value].reset_index(drop = True)

    
    def buildTree(self, df): 
        Class = df.keys()[-1]
        node = self.find_winner(df) # Find the attribute that maximizes the information gain
        attValue = np.unique(df[node]) # Get distinct value of the attribute
        tree = {}
        tree[node] = {}

        for value in attValue:
            subtable = self.get_subtable(df, node,value)
            clValue, counts = np.unique(subtable[subtable.keys()[-1]], return_counts = True)

            if (len(counts) == 1):# Subset is pure, stop generating the tree from this path
                tree[node][value] = clValue[0]
                
            else:        
                tree[node][value] = self.buildTree(subtable) # Subset is not pure, so call the function recursively
                
        return tree
    
    
    def train(self):
        self.tree = self.buildTree(self.df)
    
    
    def predict_iteration(self, tree_dict, test_column):
        firstStr = list(tree_dict.keys())[0] # Find the attribute of the first node
        secondDict = tree_dict[firstStr] # Find the subtree of this node
        featureIndex = self.label.index(firstStr) # Find the subset of that subtree

        for key in secondDict.keys():
            if(test_column[featureIndex] == key):
                if(isinstance(secondDict[key], dict)): # If this node is a dict, this node is a inner node, so call the function recursively.
                    classlabel = self.predict_iteration(secondDict[key], test_column)

                else: # This node is a leaf, just output the result.
                    classlabel = secondDict[key]
                    
        return classlabel
    
    
    def predict(self, tree_dict, test_data):
        predicted_labels = list()
        
        for i in range(test_data.shape[0]):
            predicted_labels.append(self.predict_iteration(tree_dict, test_data[i,:])) # Evaluate each line.
            
        return predicted_labels

Below is the dataset which we'll use to train the model. We put it into a dataframe named `df`.

In [3]:
training_set = np.array([['较高','是','是','否','感冒'],
                         ['非常高','否','否','否','不感冒'],
                         ['非常高','是','否','是','感冒'],
                         ['正常','是','是','是','感冒'],
                         ['正常','否','否','是','不感冒'],
                         ['较高','是','否','否','感冒'],
                         ['较高','是','否','是','感冒'],
                         ['非常高','是','是','否','感冒'],
                         ['较高','否','是','是','感冒'],
                         ['正常','是','否','否','不感冒'],
                         ['正常','是','否','是','感冒'],
                         ['正常','否','是','是','感冒'],
                         ['较高','否','否','否','不感冒'],
                         ['非常高','否','是','否','感冒'],
                         ['非常高','否','是','否','感冒'],
                         ['较高','否','否','是','感冒']])

labels = np.array(['体温','流鼻涕','肌肉疼','头疼','感冒'])

df = pd.DataFrame(training_set, columns = labels)

Now we train an ID3 classification tree model `decision_tree`:

In [4]:
decision_tree = ID3_Classification_Tree(df)
decision_tree.train()

We can see the decision tree by calling `print(decision_tree.tree)`, the result will be a dict:

In [5]:
T = decision_tree.tree
print(T)

{'肌肉疼': {'否': {'流鼻涕': {'否': {'体温': {'正常': '不感冒', '较高': {'头疼': {'否': '不感冒', '是': '感冒'}}, '非常高': '不感冒'}}, '是': {'体温': {'正常': {'头疼': {'否': '不感冒', '是': '感冒'}}, '较高': '感冒', '非常高': '感冒'}}}}, '是': '感冒'}}


Note: we can use the python package **graphviz** to visualize a decision tree and output a pdf file. Those who are interested can find it here:

https://github.com/xflr6/graphviz

We can predict new data by calling `decision_tree.predict(t, test_set)`, where T is the tree, and test_set is the new dataset:

In [6]:
test_set = np.array([['较高','否','否','否'], ['非常高','否','是','否']])
decision_tree.predict(T, test_set)

['不感冒', '感冒']

### (5) Drawbacks of ID3
* ID3 tends to choose an attribute that has more values, since these attributes would always have a larger information gain. However this is not always the best plan, so we'll introduce C4.5 algorithm in the second half.
* ID3 trees are easy to overfit, so we need prunning techniques in the second half.
* ID3 cannot deal with regression tasks or continuous variables, so we'll introduce CART algorithm in the second half.

## References:

1.机器学习 - 周志华

2.Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow, 2nd Edition - Aurélien Géron

3.统计学习方法（第2版）- 李航

4.ID3 algorithm - Wikipedia

5.数据挖掘十大算法（一）：决策树算法 python和sklearn实现 - CSDN

https://blog.csdn.net/qq_36523839/article/details/81408326

6.Decision Trees from Scratch Using ID3 Python: Coding It Up !!

https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4

7.python：从零散的字典组装成树状嵌套字典 - CSDN

https://blog.csdn.net/qq_17065591/article/details/107528137

8.Python嵌套字典的遍历 - CSDN

https://blog.csdn.net/Tw_light/article/details/104961524